<a href="https://colab.research.google.com/github/carlosjesus23/LLMs/blob/main/gsi073_aula0_regressao_logistica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GSI073 - Tópicos Especiais de Inteligência Artificial

## Definição dos dados

In [ ]:
import torch
import torch.nn as nn
import sklearn.datasets
from sklearn.model_selection import train_test_split
import numpy as np # Importar, se ainda não estiver

# 1. Carregar dados
iris = sklearn.datasets.load_iris()
X_np = iris.data
y_np = (iris.target == 1).astype(float)

# 2. Preparar dados para pytorch
X = torch.tensor(X_np, dtype=torch.float32)
y = torch.tensor(y_np, dtype=torch.float32).view(-1, 1)

# 2.1. Separar dados em Treino e Teste (Esta linha retorna ARRAYS DO NUMPY)
X_train_np, X_test_np, y_train_np, y_test_np = train_test_split(
    X_np, y_np, test_size=0.2, random_state=42
)

# 2.2. NOVO PASSO: Converter os arrays do numpy de volta para tensores PyTorch
X_train = torch.tensor(X_train_np, dtype=torch.float32)
X_test = torch.tensor(X_test_np, dtype=torch.float32)
y_train = torch.tensor(y_train_np, dtype=torch.float32).view(-1, 1)
y_test = torch.tensor(y_test_np, dtype=torch.float32).view(-1, 1)

## Definição do modelo e treinamento

In [ ]:
# 3. Definir modelo: regressão logística
modelo = nn.Linear(4, 1)  # 4 features → 1 saída (probabilidade de ser Versicolor)

# 4. Definir função de perda e algoritmo de otimização
funcao_perda = torch.nn.BCEWithLogitsLoss()  # combinação de sigmoid + BCE
optimizer = torch.optim.SGD(modelo.parameters(), lr=0.1)

## Execução do treinamento

In [ ]:
# 5. Treino
for epoch in range(1000):
    optimizer.zero_grad() # reseta gradiente senão acumula
    outputs = modelo(X_train)
    loss = funcao_perda(outputs, y_train)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Época [{epoch+1}/100], Loss: {loss.item():.4f}")

## Execução da Avaliação do Modelo

In [ ]:
# 6. Avaliação do Modelo (Usando os dados de TESTE)
with torch.no_grad(): # Desliga o cálculo de gradientes para economizar memória e tempo
    # 1. Obter os logits (saídas não normalizadas)
    test_logits = modelo(X_test)

    # 2. Converter logits para probabilidades (aplicando a sigmoid)
    test_probabilities = torch.sigmoid(test_logits)

    # 3. Converter probabilidades para classes binárias (0 ou 1)
    # 0.5 é o limiar (threshold) padrão
    test_predictions = (test_probabilities >= 0.5).float()

    # 4. Calcular a acurácia
    correct = (test_predictions == y_test).sum().item()
    total = y_test.size(0)
    accuracy = correct / total

    print(f"\n✅ Acurácia nos dados de Teste: {accuracy * 100:.2f}%")